# Parse and aggregate singleM and sourmash results for metagenome x species reads.

Questions we are asking:

## Q1: is there a lot of Artiodactyla contamination in the reads that map to pangenomes? 

Logic: suppose the pangenome contains cryptic Artiodactyla sequence (Artiodactyla is the order that includes Sus scrofa). This sequence wouldn't be directly recognized if it's not in the reference database, but the reads containing the cryptic sequence might also contain recognizable sequence.

Approach: So, we took the reads that map to the pangenome and sketched them at k=51, and searched them against everything. This notebook processes the results by metagenome x species, and (tries to) summarize them meaningfully.

## Q2: How taxonomically "pure" are the bacterial pangenomes?

Logic: suppose the pangenome for a species contains a bunch of contigs that belong to _other_ bacterial species. Then some of the reads that map to the pangenome would map to those other species. This certainly occurs to some extent, but how bad is it?

Approach: Take the reads that map to the pangenome and run SingleM on them. Turn the SingleM profile into a JSON file and process it, looking for matches (and mismatches) to the expected species.

In [1]:
import sys
import argparse
import os
import json
from collections import defaultdict
import pprint

import polars as pl

import taxburst
from taxburst import checks, tree_utils

import glob

ranks = ('superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species')


In [2]:
NOT_EXIST = 1                 # no species/genus level results
MISMATCH = 2                  # dominant family or above mismatch; OR genus and species mismatch
CORRECT_SPECIES_DOM = 3       # correct species is dominant in the read
CORRECT_SPECIES_NONDOM = 4    # correct species is in the reads, but is not dominant
CORRECT_GENUS_MATCH = 5       # correct species not there, but genus is correct

match_types = { NOT_EXIST: 'noexist', MISMATCH: 'mismatch', CORRECT_SPECIES_DOM: 'dom',
               CORRECT_SPECIES_NONDOM: 'nondom', CORRECT_GENUS_MATCH: 'genus' }

gtdb_tax_df = pl.read_csv('../inputs.mapping/names-lineages.csv')


def process_singlem(jsonfile, tax_df, *, verbose=False):
    "process SingleM -> taxburst JSON output, return match type and dominant taxonomic lineage"
    this_species = os.path.basename(jsonfile).split('.')[2]
    
    taxrow = (tax_df.row(named=True,
                         by_predicate=(pl.col("species") == this_species)))
    
    with open(jsonfile, 'rb') as fp:
        tree = json.load(fp)
    nodes = checks.collect_all_nodes(tree)
    if verbose:
        print('===')
    
    rank_vals = {}
    for rank in ranks:
        tracking = defaultdict(int)
        for node in nodes:
            name = node["name"]
            assert name         # don't allow names to be empty             
            assert node["rank"]
    
            if node["rank"] == rank:
                tracking[node["name"]] += node["count"]
    
        if not tracking:
            rank_vals[rank] = (None, None, None)
            continue
    
        total_count = sum(tracking.values())
        tracking = list(sorted(tracking.items(), key=lambda x: -x[1]))
    
        name, frac = tracking[0]
        dominance = frac / total_count
    
        rank_vals[rank] = (name, dominance, name == taxrow[rank])


    # first, check if mismatch at or above family.                          
    for check_rank in ('superkingdom', 'phylum', 'class', 'order', 'family'):
        (name, dom, is_match) = rank_vals[check_rank]
        if name is None:
            #print(f'NO MATCH at {check_rank} for {this_species}')          
            return NOT_EXIST, rank_vals
        if not is_match:
            print(f'MISMATCH: {taxrow[check_rank]} != {name} for {this_species}')
            return MISMATCH, rank_vals


    # second, check match at species:                                       
    check_rank = "species"
    (name, dom, is_match) = rank_vals[check_rank]
    if name is None:        # skip                                          
        pass
    elif is_match:
        if dom >= 0.95:
            return CORRECT_SPECIES_DOM, rank_vals
        else:
            return CORRECT_SPECIES_NONDOM, rank_vals
    else:
        # is genus a match?                                                 
        (genus_name, genus_dom, genus_is_match) = rank_vals["genus"]
        if genus_is_match: # and genus_dom >= 0.95: 
            return CORRECT_GENUS_MATCH, rank_vals
        else:
            print(f'MISMATCH_SP: {taxrow[check_rank]} != {name} for {this_species}')   
            return MISMATCH, rank_vals

    return NOT_EXIST, rank_vals



In [3]:
# load all available singleM outputs and create a dataframe with the match types
track_xx = []
for jsonfile in glob.glob('../outputs.mapping/bams.rand/*.profile.json'):
    matchtype, rank_vals = process_singlem(jsonfile, gtdb_tax_df)
    basename = os.path.basename(jsonfile)
    acc, _, species, _ = basename.split('.', 3)
    #print(acc, species)
    d = dict(acc=acc, species=species, match_type=match_types[matchtype])
    track_xx.append(d)

singlem_df = pl.DataFrame(track_xx)

MISMATCH_SP: s__JAFBIX01 sp021531895 != s__Dorea formicigenerans for s__JAFBIX01 sp021531895
MISMATCH_SP: s__Roseburia inulinivorans != s__Agathobacter sp000434275 for s__Roseburia inulinivorans
MISMATCH_SP: s__Roseburia inulinivorans != s__Agathobacter sp000434275 for s__Roseburia inulinivorans
MISMATCH_SP: s__Holdemanella porci != s__Floccifex porci for s__Holdemanella porci
MISMATCH_SP: s__Roseburia inulinivorans != s__UBA2868 sp004552595 for s__Roseburia inulinivorans
MISMATCH_SP: s__Phocaeicola vulgatus != s__Prevotella sp945863825 for s__Phocaeicola vulgatus
MISMATCH_SP: s__Roseburia inulinivorans != s__Agathobacter sp000434275 for s__Roseburia inulinivorans
MISMATCH_SP: s__Roseburia inulinivorans != s__Agathobacter sp000434275 for s__Roseburia inulinivorans
MISMATCH_SP: s__JALFVM01 sp022787145 != s__Clostridium_AP scindens for s__JALFVM01 sp022787145
MISMATCH: f__Muribaculaceae != f__Bacteroidaceae for s__Sodaliphilus sp004557565
MISMATCH: f__Muribaculaceae != f__Bacteroidaceae 

In [4]:
# (there should be 1600 of them)
assert singlem_df['match_type'].value_counts().sort(by='count')['count'].sum() == 1600
singlem_df['match_type'].value_counts().sort(by='count')


match_type,count
str,u32
"""dom""",126
"""noexist""",197
"""genus""",218
"""nondom""",402
"""mismatch""",657


In [5]:
def process_gather_lineage(csvfile):
    """Process sourmash gather outputs.
    
    Pull all artiodactyla and some their unique intersect bp; return df, total bp, total weighted bp.
    """
    df = pl.read_csv(csvfile)
    df = df.filter(pl.col("lineage").str.starts_with('Eukaryota;Chordata;Mammalia;Artiodactyla'))

    if df.is_empty():
        return df, 0, 0

    total = df['unique_intersect_bp'].sum()
    total_weight = df['n_unique_weighted_found'].sum() * df['scaled'].unique().to_list()[0]

    return df, total, total_weight


In [6]:
# find an example none-empty one, just for debugging
for csvfile in glob.glob('../outputs.mapping/bams.rand/*.with-lineages.csv'):
    df, total, total_weight = process_gather_lineage(csvfile)
    if not df.is_empty(): break

print(total, total_weight)
df

10000 10000


intersect_bp,f_orig_query,f_match,f_unique_to_query,f_unique_weighted,average_abund,median_abund,std_abund,filename,name,md5,f_match_orig,unique_intersect_bp,gather_result_rank,remaining_bp,query_filename,query_name,query_md5,query_bp,ksize,moltype,scaled,query_n_hashes,query_abundance,query_containment_ani,match_containment_ani,average_containment_ani,max_containment_ani,potential_false_negative,n_unique_weighted_found,sum_weighted_found,total_weighted_hashes,lineage
i64,f64,f64,f64,f64,f64,f64,f64,str,str,str,f64,i64,i64,i64,str,str,str,i64,i64,str,i64,i64,bool,f64,f64,f64,f64,bool,i64,i64,i64,str
10000,0.000291,0.000004,0.000291,0.000056,1.0,1.0,0.0,"""/group/ctbrowngrp5/sourmash-db…","""GCF_000003025.6 pig (Sus scrof…","""2edac3821d65091319e98478f22e94…",0.000004,10000,504,17950000,"""outputs.mapping/bams.rand/SRR8…","""SRR8960391.x.s__Gemmiger qucib…","""a34f542d""",34370000,51,"""DNA""",10000,3437,true,0.852439,0.784714,0.818576,0.852439,true,1,14497,17780,"""Eukaryota;Chordata;Mammalia;Ar…"


In [7]:
# load all available sourmash outptus => `smash_df`
track_xx = []
for csvfile in glob.glob('../outputs.mapping/bams.rand/*.with-lineages.csv'):
    df, total, total_weight = process_gather_lineage(csvfile)
    basename = os.path.basename(csvfile)
    acc, _, species, _ = basename.split('.', 3)
    #print(acc, species, len(df))

    d = dict(acc=acc, species=species, euk_bp=total, euk_bp_weighted=total_weight)
    track_xx.append(d)

smash_df = pl.DataFrame(track_xx)

In [8]:
# expect 1600
assert len(smash_df) == 1600
smash_df

acc,species,euk_bp,euk_bp_weighted
str,str,i64,i64
"""SRR8960478""","""s__JALFVM01 sp022787145""",0,0
"""ERR1135282""","""s__Cryptobacteroides sp9005469…",0,0
"""SRR11183981""","""s__Sodaliphilus sp004557565""",0,0
"""ERR1135289""","""s__Gemmiger qucibialis""",0,0
"""SRR8960391""","""s__Gemmiger qucibialis""",10000,10000
…,…,…,…
"""SRR8960379""","""s__UBA2868 sp004552595""",0,0
"""SRR15057929""","""s__UBA2868 sp004552595""",0,0
"""SRR8960492""","""s__Sodaliphilus sp004557565""",240000,570000


In [9]:
# awesome, join the dataframes to produce a single main dataframe that 
# contains metagenome x species summaries of both sourmash and singleM output
join_df = smash_df.join(singlem_df, on=['acc', 'species'], how='inner')
join_df.sort(by='euk_bp_weighted', descending=True)

acc,species,euk_bp,euk_bp_weighted,match_type
str,str,i64,i64,str
"""SRR14369134""","""s__Sodaliphilus sp004557565""",2670000,11830000,"""nondom"""
"""SRR11489750""","""s__Sodaliphilus sp004557565""",1220000,6500000,"""mismatch"""
"""SRR14369134""","""s__Lactobacillus amylovorus""",3540000,6060000,"""genus"""
"""SRR14369134""","""s__Phocaeicola vulgatus""",4060000,4890000,"""mismatch"""
"""SRR14369268""","""s__Sodaliphilus sp004557565""",1510000,4610000,"""mismatch"""
…,…,…,…,…
"""ERR1135289""","""s__Phocaeicola vulgatus""",0,0,"""mismatch"""
"""SRR8655115""","""s__Phascolarctobacterium_A suc…",0,0,"""dom"""
"""SRR2329775""","""s__Bariatricus sp004560705""",0,0,"""nondom"""


## Summarize

In [10]:
best_species = ['s__Mogibacterium_A kristiansenii',
                's__Phascolarctobacterium_A succinatutens',
                's__Cryptobacteroides sp900546925',
                's__Holdemanella porci']

summary = []
#for species in best_species:
for species in join_df['species'].unique():

    sub_df = join_df.filter(pl.col("species") == species)
    assert len(sub_df) == 100
    dd = {}
    for d in sub_df['match_type'].value_counts().to_dicts():
        dd[d['match_type']] = d['count']

    euk_bp = sum(sub_df['euk_bp'])
    euk_bp_weighted = sum(sub_df['euk_bp_weighted'])
    euk_freq = len(sub_df.filter(pl.col("euk_bp") > 10_000))
    summary.append((euk_bp, euk_bp_weighted, species, dd, euk_freq / len(sub_df)))
    #print(species, dd)
    #print(sum(sub_df['euk_bp']) / 10_000)

print('** summary by species, sorted by amount of estimate pig overlap:\n')
summary.sort()
for euk_bp, euk_bp_weighted, species, dd, euk_freq in summary:
    s = " ".join(f"{k}={v}" for (k, v) in sorted(dd.items()) )
    print(f"{euk_bp:>9d} {euk_bp_weighted:>9d}  {euk_freq:.2f} {species:<41} {s}")

print('')
print('** summary by species, sorted by number of singleM mismatches:\n')
summary.sort(key=lambda x: x[3].get('mismatch', 0))
for euk_bp, euk_bp_weighted, species, dd, euk_freq in summary:
    s = " ".join(f"{k}={v}" for (k, v) in sorted(dd.items()) )
    print(f"{euk_bp:>9d} {euk_bp_weighted:>9d}  {euk_freq:.2f} {species:<41} {s}")

** summary by species, sorted by amount of estimate pig overlap:

    20000     40000  0.00 s__Fimisoma sp002320005                   dom=12 mismatch=39 noexist=20 nondom=29
    80000    100000  0.01 s__JALFVM01 sp022787145                   dom=1 mismatch=87 noexist=8 nondom=4
   480000    910000  0.08 s__UBA2868 sp004552595                    dom=14 mismatch=56 noexist=3 nondom=27
  1630000   1730000  0.19 s__Bariatricus sp004560705                dom=2 mismatch=55 noexist=5 nondom=38
  1740000   1870000  0.19 s__Cryptobacteroides sp900546925          dom=2 genus=35 mismatch=1 noexist=11 nondom=51
  1900000   2020000  0.19 s__Prevotella sp002251295                 dom=6 genus=40 mismatch=5 noexist=6 nondom=43
  2390000   2540000  0.23 s__Roseburia inulinivorans                mismatch=92 noexist=4 nondom=4
  2440000   2730000  0.21 s__Phascolarctobacterium_A succinatutens  dom=21 genus=31 noexist=4 nondom=44
  3270000   3830000  0.23 s__Mogibacterium_A kristiansenii          dom=48 m

## Some interim conclusions

These four have few mismatches (good taxonomic purity), and relatively little pig sequence:
* s__Phascolarctobacterium_A succinatutens
* s__Cryptobacteroides sp900546925
* s__Mogibacterium_A kristiansenii
* s__Prevotella sp002251295

These have relatively few mismatches, and more pig sequence:
* s__Prevotella sp000434975
* s__Holdemanella porci

Not great:
* s__Fimisoma sp002320005 - some mismatches, very little pig
* s__Roseburia inulinivorans - _many_ mismatches, not much pig
* s__Gemmiger qucibialis - _many_ mismatches, lotsa pig
* s__Bariatricus sp004560705 - _many_ mismatches
* s__UBA2868 sp004552595 - many mismatches

LOTS of mismatches:
* s__JALFVM01 sp022787145 - _many_ mismatches
* s__JAFBIX01 sp021531895

Terrible:
* s__Phocaeicola vulgatus - MANY mismatches, lotsa pig
* s__Lactobacillus amylovorus - _many_ mismatches, lots of pig
* s__Sodaliphilus sp004557565
